In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

import os
print(os.getcwd()) 

X = pd.read_csv('X_features.csv', index_col=0)
y = pd.read_csv('y_features.csv', index_col=0)
groups = X.pop('Feature_7')

test = pd.read_csv('test_features.csv', index_col=0).drop(columns = 'Feature_7')
sample = pd.read_csv('sample_submission_2.csv')
df_train = pd.read_csv('train.csv').set_index('Id')
df_test = pd.read_csv('test_2.csv').set_index('Id')
weights = df_train.loc[:,'Weight_Intraday':'Weight_Daily']

C:\Users\matze\Documents\Winton\CRISPDM


In [15]:
#Helper
def typecasting(df):
    categorial_cols = ['Feature_20','Feature_5']
    binary_cols = ['Feature_16']

    for col in categorial_cols:
        df[col] = df[col].astype('category')
    
    for col in binary_cols:
        df[col] = (df[col]-1).astype('category')

def trainscore(y_true,y_pred,weights):
    weights = weights.loc[y_true.index,:]
    daily = pd.concat([weights.Weight_Daily]*2,axis=1)
    minute = pd.concat([weights.Weight_Intraday]*60,axis=1)
    weights = pd.concat([minute,daily],axis=1)
    weights.columns = df_train.loc[:,'Ret_121':'Ret_PlusTwo'].columns
    
    
    wmea = (((abs(y_true-y_pred))*weights).values.flatten().sum()/y_true.size)
    
    
    #minute_ret = abs(y_true.loc[:,'Ret_121':'180']-y_pred.loc[:,'Ret_121':'Ret_180'])*weights.Weight_Intraday
    return wmea


def create_core_features(train,test):
    X_train=train.loc[:,'Feature_1':'Ret_MinusOne']
    X_val=test.loc[:,'Feature_1':'Ret_MinusOne']
    from sklearn.preprocessing import Imputer

    typecasting(X_train)
    X_train = pd.get_dummies(X_train,drop_first=True)

    typecasting(X_val)
    X_val = pd.get_dummies(X_val,drop_first=True)

    imputer = Imputer(strategy='mean', axis=0)
    X_train = pd.DataFrame(imputer.fit_transform(X_train), index= X_train.index,columns=X_train.columns)
    X_val = pd.DataFrame(imputer.fit_transform(X_val), index= X_val.index,columns=X_val.columns)
    X_train['Ret_MinutePast'] = train.loc[:,'Ret_2':'Ret_120'].sum(axis=1)
    X_val['Ret_MinutePast'] = test.loc[:,'Ret_2':'Ret_120'].sum(axis=1)
    return X_train, X_val

### Creating featuresets

In [17]:
core_features, core_features_test = create_core_features(df_train,df_test)
# generated features is X
#Adding the interactions to Featureset
crafted_features = pd.read_csv('BA_Features/X_features_BA.csv', index_col=0)
crafted_features_test = pd.read_csv('BA_Features/test_features_BA.csv', index_col=0)

In [21]:
crafted_features['interaction_3']= crafted_features.PR_120 * crafted_features.grouped_mad_fet_7
crafted_features_test['interaction_3']= crafted_features_test.PR_120 * test.grouped_mad_fet_7

#X['interaction_4']= X.smoothed_minute_mean * X.minute_mad
#test['interaction_4']= test.smoothed_minute_mean * test.minute_mad

crafted_features['interaction_5']= crafted_features.minute_mad * crafted_features.PR_120
crafted_features_test['interaction_5']= crafted_features_test.minute_mad * crafted_features_test.PR_120

crafted_features['interaction_6']= crafted_features.smoothed_minute_mean * crafted_features.grouped_mad_fet_5
crafted_features_test['interaction_6']= crafted_features_test.smoothed_minute_mean * crafted_features_test.grouped_mad_fet_5

crafted_features['interaction_7']= crafted_features.PR_120 * crafted_features.fet_7_Minute_MAD
crafted_features_test['interaction_7']= crafted_features_test.PR_120 * crafted_features_test.fet_7_Minute_MAD

crafted_features['interaction_8']= crafted_features.fet_7_Minute_MAD * crafted_features.fet_7_Minute_Mean
crafted_features_test['interaction_8']= crafted_features_test.fet_7_Minute_MAD * crafted_features_test.fet_7_Minute_Mean

print(crafted_features.shape,crafted_features_test.shape)

(40000, 102) (120000, 102)


In [50]:
all_features = pd.concat([core_features,crafted_features],axis=1).drop('Feature_7',axis='columns')
all_features_test = pd.concat([core_features_test,crafted_features_test],axis=1).drop('Feature_7',axis='columns')

features_and_target = all_features.copy()
features_and_target['Ret_PlusOne'] = y.Ret_PlusOne
features_and_target['Ret_PlusTwo'] = y.Ret_PlusTwo
features_and_target = features_and_target


In [ ]:
import time
t0=time.time()
corr_spearman = features_and_target.corr(method='spearman').to_csv('BA_Features/spearman_cor.csv')
print("time:", round(time.time()-t0, 3), "s")
corr_pearson = features_and_target.corr(method='pearson').to_csv('BA_Features/pearson_cor.csv')
print("time:", round(time.time()-t0, 3), "s")

In [89]:
abs(corr_spearman.Ret_PlusOne).sort_values(ascending=False)[1:20]

interaction_8                 0.065062
fet_7_Minute_Mean             0.060549
interaction_5                 0.038164
interaction_3                 0.037041
interaction_7                 0.036667
Delta_last_5                  0.036181
interaction_6                 0.036009
smoothed_minute_sum           0.035779
smoothed_minute_mean          0.035779
interaction_1                 0.035623
PR_Last_10                    0.035519
Delta_119                     0.035339
PR_120                        0.035120
Ret_MinutePast                0.033987
Delta_120                     0.033637
minute_sum                    0.033637
minute_mean                   0.033637
Delta_last_60                 0.032584
fet_7_absdiff_RetDaily_Mad    0.032109
Name: Ret_PlusOne, dtype: float64

In [90]:
abs(corr_pearson.Ret_PlusOne).sort_values(ascending=False)[1:20]

interaction_2           0.133803
interaction_3           0.111315
PR_Last_30              0.108404
interaction_8           0.102100
PR_Last_60              0.098852
interaction_7           0.098168
last_minute_diff        0.098035
Delta_last_60           0.096432
fet_7_Minute_Mean       0.095669
PR_Last_10              0.092923
interaction_5           0.091822
Delta_119               0.091589
abs_last_minute_diff    0.088304
Delta_last_10           0.088294
interaction_6           0.084366
smoothed_minute_mean    0.084310
smoothed_minute_sum     0.084310
Ret_MinutePast          0.084249
minute_mean             0.083243
Name: Ret_PlusOne, dtype: float64

In [95]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
n=40000

t0 = time.time()
mic_Ret_PlusOne = mutual_info_regression(all_features.sample(n,random_state=0), y.Ret_PlusOne.sample(n,random_state=0), random_state=0)
print('Time: ',time.time()-t0)
t0 = time.time()
#select10 = SelectKBest(score_func=mutual_info_regression, k=20).fit(all_features.sample(n,random_state=0), y.Ret_PlusOne.sample(n,random_state=0))
print('Time: ',time.time()-t0)

Time:  3044.978446006775
Time:  0.0


In [ ]:
#mic_Ret_PlusTwo = mutual_info_regression(all_features.sample(n,random_state=0), y.Ret_PlusOne.sample(n,random_state=0), random_state=0)

In [102]:
pd.DataFrame(mic_Ret_PlusOne, index = all_features.columns.values,columns=['MIC_PlusOne']).sort_values('MIC_PlusOne',ascending=False).to_csv('BA_Features/MIC_PlusOne.csv')

In [104]:
pd.DataFrame(mic_Ret_PlusOne, index = all_features.columns.values,columns=['MIC_PlusOne']).sort_values('MIC_PlusOne',ascending=False)

,MIC_PlusOne
minute_std,0.134975
minute_sem,0.134975
smoothed_minute_std,0.134404
smoothed_minute_sem,0.134404
minute_var,0.131280
smoothed_minute_var,0.129490
minute_median,0.127389
minute_mad,0.127389
MAD_120,0.127389
smoothed_minute_mad,0.125025


In [79]:
all_features.columns[select10.get_support(indices=True)].tolist()

['MAD_120',
 'MAD_last_60',
 'MAD_first_60',
 'MAD_last_30',
 'MAD_last_15',
 'fet_7_RetMinTwo_Mad',
 'smoothed_minute_mad',
 'smoothed_minute_median',
 'smoothed_minute_std',
 'smoothed_minute_var',
 'smoothed_minute_sem',
 'minute_mad',
 'minute_median',
 'minute_std',
 'minute_var',
 'minute_sem',
 'minute_25_quantile',
 'minute_75_quantile',
 'fet_7_Minute_MAD',
 '0']

## Predicting Ret_PlusOne

### Ridge

In [94]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupShuffleSplit
group_shuffle = GroupShuffleSplit(n_splits=10, test_size=0.60, train_size=0.4, random_state=0)
from sklearn.linear_model import Ridge

from sklearn.feature_selection import RFE


train = all_features
test = all_features_test
cols_to_use = ['Ret_MinusOne']
learner = Ridge()
param_grid= {'alpha': [20,19,18,17,16,15,12,11],'normalize':[False, True],'max_iter':[700]}


gs= GridSearchCV(learner, param_grid, iid=False, refit=True, cv=group_shuffle, return_train_score =True, scoring='neg_mean_absolute_error')
gs.fit(X[cols_to_use], y.Ret_PlusOne, groups)
gs.cv_results_ 
display(gs.best_estimator_)
display(gs.best_score_ )

#Trainscore
grid = pd.DataFrame(np.zeros([40000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns,index=df_train.index)
grid.shape
print("Zero Benchmark: ",trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5) )
grid['Ret_PlusOne'] = ((gs.predict(train[cols_to_use])))
print("Validation Benchmark: ",trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5) )


###Submission
grid = pd.DataFrame(np.zeros([120000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns)
grid.shape

grid['Ret_PlusOne'] = ((gs.predict(test[cols_to_use])))

sample['Predicted'] = grid.values.flatten()
#sample.to_csv('submissions/featureset_RFE_20_2.csv', index=False)

Ridge(alpha=12, copy_X=True, fit_intercept=True, max_iter=700, normalize=True,
   random_state=None, solver='auto', tol=0.001)

-0.015770296333707377

Zero Benchmark:  1773.9244
Validation Benchmark:  1773.83633


In [ ]:
#rfe
n = 20

t0=time.time()
rfe = RFE(gs.best_estimator_,n_features_to_select=n, step=1, verbose=1)
rfe.fit(train, y.Ret_PlusOne)
print("training time:", round(time.time()-t0, 3), "s")

cols_to_use = train.columns[rfe.get_support(indices=True)]
gs= GridSearchCV(gbm, param_grid, iid=False, refit=True, cv=group_shuffle, return_train_score =True, scoring='neg_mean_absolute_error')
gs.fit(X[cols_to_use], y.Ret_PlusOne, groups)
gs.cv_results_ 
display(gs.best_estimator_)
display(gs.best_score_ )



grid = pd.DataFrame(np.zeros([120000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns)
grid.shape

grid['Ret_PlusOne'] = ((gs.predict(test[cols_to_use])))

sample['Predicted'] = grid.values.flatten()
sample.to_csv('submissions/GBM_featureset_RFE_20.csv', index=False)

### Huber

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupShuffleSplit
group_shuffle = GroupShuffleSplit(n_splits=10, test_size=0.60, train_size=0.4, random_state=0)
from sklearn.linear_model import Huber

from sklearn.feature_selection import RFE
#(estimator, n_features_to_select=None, step=1, verbose=0)


train = all_features
test = all_features_test
cols_to_use = ['Ret_MinusOne']
learner = Huber()
param_grid= {'alpha': [100,20,19,18,17,16,15,12,11,1,0.1],'epsilon':[1,1.35,1.8],'max_iter':[700]}


gs= GridSearchCV(learner, param_grid, iid=False, refit=True, cv=group_shuffle, return_train_score =True, scoring='neg_mean_absolute_error')
gs.fit(X[cols_to_use], y.Ret_PlusOne, groups)
gs.cv_results_ 
display(gs.best_estimator_)
display(gs.best_score_ )

#Trainscore
grid = pd.DataFrame(np.zeros([40000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns,index=df_train.index)
grid.shape
print("Zero Benchmark: ",trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5) )
grid['Ret_PlusOne'] = ((gs.predict(train[cols_to_use])))
print("Validation Benchmark: ",trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5) )


###Submission
grid = pd.DataFrame(np.zeros([120000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns)
grid.shape

grid['Ret_PlusOne'] = ((gs.predict(test[cols_to_use])))

sample['Predicted'] = grid.values.flatten()
#sample.to_csv('submissions/featureset_RFE_20_2.csv', index=False)

### Random Forrest

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupShuffleSplit
group_shuffle = GroupShuffleSplit(n_splits=10, test_size=0.60, train_size=0.4, random_state=0)
from sklearn.ensemble import RandomForestRegressor

train = all_features
test = all_features_test
cols_to_use = ['Ret_MinusOne']
learner = RandomForestRegressor()
param_grid= {'bootstrap': [True, False],#kein plan also auch weg
             'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],#tiefer
             'max_features': ['auto', 'sqrt'],#wahrscheinlich weniger glaub kürzere fitting time
             'min_samples_leaf': [1, 2, 4], #höher
             'min_samples_split': [2, 5, 10],#kann wahrscheinlich weg
             'n_estimators': [200, 400, 600]}#mehr bedeuten dauert länger


gs= GridSearchCV(learner, param_grid, iid=False, refit=True, cv=group_shuffle, return_train_score =True, scoring='neg_mean_absolute_error')
gs.fit(X[cols_to_use], y.Ret_PlusOne, groups)
gs.cv_results_ 
display(gs.best_estimator_)
display(gs.best_score_ )

#Trainscore
grid = pd.DataFrame(np.zeros([40000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns,index=df_train.index)
grid.shape
print("Zero Benchmark: ",trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5) )
grid['Ret_PlusOne'] = ((gs.predict(train[cols_to_use])))
print("Validation Benchmark: ",trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5) )


###Submission
grid = pd.DataFrame(np.zeros([120000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns)
grid.shape

grid['Ret_PlusOne'] = ((gs.predict(test[cols_to_use])))

sample['Predicted'] = grid.values.flatten()
#sample.to_csv('submissions/featureset_RFE_20_2.csv', index=False)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupShuffleSplit
group_shuffle = GroupShuffleSplit(n_splits=10, test_size=0.60, train_size=0.4, random_state=0)
from sklearn.ensemble import GradientBoostRegressor

train = all_features
test = all_features_test
cols_to_use = ['Ret_MinusOne']
learner = GradientBoostRegressor()
param_grid= {'bootstrap': [True, False],#kein plan also auch weg
             'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],#tiefer
             'max_features': ['auto', 'sqrt'],#wahrscheinlich weniger glaub kürzere fitting time
             'min_samples_leaf': [1, 2, 4], #höher
             'min_samples_split': [2, 5, 10],#kann wahrscheinlich weg
             'n_estimators': [200, 400, 600]}#mehr bedeuten dauert länger


gs= GridSearchCV(learner, param_grid, iid=False, refit=True, cv=group_shuffle, return_train_score =True, scoring='neg_mean_absolute_error')
gs.fit(X[cols_to_use], y.Ret_PlusOne, groups)
gs.cv_results_ 
display(gs.best_estimator_)
display(gs.best_score_ )

#Trainscore
grid = pd.DataFrame(np.zeros([40000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns,index=df_train.index)
grid.shape
print("Zero Benchmark: ",trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5) )
grid['Ret_PlusOne'] = ((gs.predict(train[cols_to_use])))
print("Validation Benchmark: ",trainscore(df_train.loc[:,'Ret_121':'Ret_PlusTwo'],grid,weights).round(5) )


###Submission
grid = pd.DataFrame(np.zeros([120000, 62]), columns = df_train.loc[:, 'Ret_121':'Ret_PlusTwo'].columns)
grid.shape

grid['Ret_PlusOne'] = ((gs.predict(test[cols_to_use])))

sample['Predicted'] = grid.values.flatten()
#sample.to_csv('submissions/featureset_RFE_20_2.csv', index=False)